# Testing Notebook for Cvlac Product Miner.
- Author: David Santiago Barreto Mora
- Last updated: 14/01/2024

---


In [ ]:
# Import Python libraries
from typing import List, NamedTuple
from enum import Enum
import traceback


In [ ]:
# Import third party modules
from bs4 import BeautifulSoup
# import knime.scripting.io as knio


In [ ]:
# Definition of Table type Enum
class TableType(Enum):
    NESTED_TABLES = "Self contained product. Product information is inside nested tables."
    BLOCKQUOTE_PRODUCTS = "Self contained product. All product information inside Blockquote"
    SUBTYPE_AS_LIST_ITEM = "Scattered information. Subtype is in <li> tag."
    SUBTYPE_AS_EMPTY_LIST_ITEM= "Scattered information. Subtype should be in <li> tag, but the tag is empty."
    SUBTYPE_AS_BOLD_TAG = "Scattered information. Subtype is in <b> tag."
